Load dataset to df

In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import datetime
import urllib
import calendar

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict


In [84]:
df=pd.read_pickle('311_Requests.pickle')

df.columns.values

array([':@computed_region_8tb6_tjh9', ':@computed_region_hrit_uzpe',
       'address', 'agency_responsible', 'expected_datetime', 'lat', 'lon',
       'media_url', 'requested_datetime', 'service_code', 'service_name',
       'service_notice', 'shape', 'status', 'updated_datetime', 'zipcode'], dtype=object)

In [85]:
list(df['service_name'].unique())

['Sanitation / Dumpster Violation',
 'Illegal Dumping',
 'Rubbish/Recyclable Material Collection',
 'Dead Animal in Street',
 'Graffiti Removal',
 'Street Defect',
 'Information Request',
 'Directory Assistance',
 'Zoning Business',
 'Abandoned Vehicle',
 'Street Trees',
 'Street Light Outage',
 'Smoke Detector',
 'Vacant House or Commercial',
 'Alley Light Outage',
 'Parks and Rec Safety and Maintenance',
 'Miscellaneous',
 'Construction Site Task Force',
 'Stop Sign Repair',
 'Complaint (Streets)',
 'Vacant Lot Clean-Up',
 'Inlet Cleaning',
 'Hydrant Request',
 'Other (Streets)',
 'Maintenance Residential or Commercial',
 'Building Construction',
 'Zoning Residential',
 'Police Complaint',
 'Building Dangerous',
 'License Residential',
 'Infestation Residential',
 'Traffic (Other)',
 'Manhole Cover',
 'Traffic Signal Emergency',
 'Dangerous Sidewalk',
 'Street Paving',
 'Abandoned Bike',
 'Boarding Room House',
 'Fire Residential or Commercial',
 'Salting',
 'Daycare Residential or C

## Top10 problems by each department for last week

In [86]:
df=df.set_index('requested_datetime')

In [87]:
df.index = df.index.sort_values(ascending=False)
today = datetime.datetime.now()

Current month and week

In [81]:
df_week=df.loc[today:today - pd.Timedelta(days=7)].reset_index()
df_cmonth=df.loc[today:today - pd.Timedelta(days=30)].reset_index()

In [31]:
df.loc[lastdayfrom - pd.Timedelta(days=30):lastdayfrom].reset_index()

datetime.datetime(2016, 9, 27, 16, 39, 34, 504929)

In [133]:
agencies=df['agency_responsible'].unique()

In [141]:
agencies = df_week.groupby(['agency_responsible']).count()['zipcode'].sort_values(ascending=False)[:15]

In [147]:
agencies

agency_responsible
License & Inspections                 832
Streets Department                    608
Community Life Improvement Program    122
Police Department                      90
Parks & Recreation                     60
First Judicial District/Courts         50
Philly311 Contact Center               39
Directory Assistance                   34
Water Department (PWD)                 27
Fire Department                        16
Revenue                                16
Department of Public Health            16
Department of Records                  14
City Council                           13
Register of Wills/ Orphans Court        9
Name: zipcode, dtype: int64

In [134]:
for i in agencies:
    print('Agency:',i)

Agency: Streets Department
Agency: Community Life Improvement Program
Agency: Philly311 Contact Center
Agency: nan
Agency: First Judicial District/Courts
Agency: Directory Assistance
Agency: License & Inspections
Agency: Department of Records
Agency: Office of Property Assessment- OPA
Agency: Water Department (PWD)
Agency: City Controller
Agency: Managing Director's Office- MDO
Agency: Probation/Parole
Agency: Register of Wills/ Orphans Court
Agency: Office of the Director of Finance
Agency: City Council
Agency: Police Department
Agency: Office of Housing & Community Development- OHCD
Agency: Department of Public Health
Agency: Office of Human Resources- OHR
Agency: Parks & Recreation
Agency: State/Federal Government Offices
Agency: Office of Arts, Culture & Creative Economy- OACCE
Agency: Revenue
Agency: Fire Department
Agency: Mayor's Office
Agency: Water Revenue
Agency: Law Department/ City Solicitor
Agency: Office of Homeless Services - OHS
Agency: City Commissioners Office
Agency:

In [6]:
df.groupby('Service Notice').count()['Zipcode'].sort_values(ascending=False)[:10]

Service Notice
None                242218
20 Business Days     28311
2 Business Days      15459
5 Business Days      14302
3 Business Days      10888
30 Business Days      7707
90 Business Days      4547
60 Business Days      3812
15 Business Days      3692
10 Business Days      3328
Name: Zipcode, dtype: int64

In [23]:
pd.to_datetime(df_ss['Updated Date/Time'])-pd.to_datetime(df_ss['Requested Date/Time'])

0       1 days 16:59:45
1       1 days 17:53:38
2       1 days 16:37:51
3       1 days 16:44:10
4       1 days 15:39:01
5       1 days 14:56:57
6       1 days 13:58:10
7       1 days 22:03:26
8       1 days 21:53:42
9       1 days 20:41:31
10      1 days 19:02:04
11      1 days 17:13:27
12      1 days 17:01:38
13      1 days 17:01:39
14      1 days 15:49:21
15      1 days 15:44:21
16      1 days 13:31:43
17      1 days 10:34:24
18      1 days 21:41:24
19      1 days 21:41:24
20      1 days 21:10:45
21      1 days 21:00:45
22      1 days 20:56:28
23      1 days 20:20:01
24      1 days 23:21:41
25      5 days 13:47:20
26      2 days 21:18:38
27      2 days 21:22:37
28      3 days 15:58:02
29      3 days 15:56:27
             ...       
470     0 days 00:00:28
471     0 days 00:00:10
472     0 days 00:00:10
473    33 days 03:06:15
474     0 days 00:12:40
475     0 days 00:03:04
476     0 days 00:00:19
477     0 days 18:05:12
478     0 days 00:00:21
479     0 days 00:01:36
480     0 days 0